#`**Requirements**`

- isobar is a Python library for creating and manipulating musical patterns, designed for use in algorithmic composition, generative music and sonification.

[Github repo](https://github.com/ideoforms/isobar)

In [16]:
!pip install isobar

In [17]:
import requests
import pprint
import isobar as iso
from isobar.io import MidiFileOutputDevice

#`Web scraping weather data`

---



Replace APIKEY with your personal key

In [18]:

APIKEY='b2cb32eb502aa18d4a5955f9b6cea4d8'
City_API_endpoint = "http://api.openweathermap.org/data/2.5/forecast?q="
Country = ",GR,"
join_key = "&appid=" + APIKEY
units = "&units=metric"

You can set the City variable to any city

To view the forecast you requested, open the printed link in a new tab.

In [19]:
City = "Patras"
city_forecast = City_API_endpoint + City + Country + join_key + units

print(City+" Forecast :", city_forecast)

Patras Forecast : http://api.openweathermap.org/data/2.5/forecast?q=Patras,GR,&appid=b2cb32eb502aa18d4a5955f9b6cea4d8&units=metric


Here is were we make use of the request package

- When you ping a website or portal for information this is called making a request. That is exactly what the Requests package has been designed to do.

- To get a webpage you would do something like the following:
`r = requests.get(‘https://github.com/timeline.json’)`

- Then we want to get the result of the response into a variable and work with it.

In [20]:
forecast_json_data = requests.get(city_forecast).json() # Make the request
#pprint.pprint(forecast_json_data )

[2021-11-11 10:10:32,073] Starting new HTTP connection (1): api.openweathermap.org:80
[2021-11-11 10:10:32,095] http://api.openweathermap.org:80 "GET /data/2.5/forecast?q=Patras,GR,&appid=b2cb32eb502aa18d4a5955f9b6cea4d8&units=metric HTTP/1.1" 200 16056


In [21]:
# Create Empty Arrays to Store Weather Data

# Main
temp_prediction = []
temp_feels_like_prediction = []
temp_min_prediction = []
temp_max_prediction = []
pressure_prediction = []
sea_level_prediction = []
grnd_level_prediction = []
humidity_prediction = []
temp_kf_prediction = []
# Wind
wind_speed_prediction = []
wind_degree_prediction = []

prediction_num = 0

# Loop Through the JSON
for num_forecasts in forecast_json_data['list']:
    
    # Main
    temp_prediction.append(forecast_json_data['list'][prediction_num]['main']['temp'])
    pressure_prediction.append(forecast_json_data['list'][prediction_num]['main']['pressure'])
    sea_level_prediction.append(forecast_json_data['list'][prediction_num]['main']['sea_level'])
    grnd_level_prediction.append(forecast_json_data['list'][prediction_num]['main']['grnd_level'])
    humidity_prediction.append(forecast_json_data['list'][prediction_num]['main']['humidity'])
    # Wind
    wind_speed_prediction.append(forecast_json_data['list'][prediction_num]['wind']['speed'])
    wind_degree_prediction.append(forecast_json_data['list'][prediction_num]['wind']['deg'])
    
    prediction_num += 1


#`**Sonification**`

##`Useful methods`

In [22]:
def scale(old_value, old_min, old_max, new_min, new_max):
  new_value = ( (old_value - old_min) / (old_max - old_min) ) * (new_max - new_min) + new_min
  return new_value

def count(iterable):
  return sum(1 for _ in iterable)

def cal_average(num):
    sum_num = 0
    for t in num:
        sum_num = sum_num + t           

    avg = sum_num / len(num)
    return avg

## `Convert pressure to a root note`
A rapid fall of air pressure is a sign for an approaching low and is a sign for bad weather. 
Depending upon the drop-rate (per hour) you may exspect strong winds up to a gale. The dependeny pressure-drop-per-hour > wind is depending upon the latitude.

We can scale pressure data to a range between 0-11 in order to determine the root key note of the sequences we are generating.



In [23]:
note_names = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
pressureMin = 1000 # mbar
pressureMax = 1083 # mbar

# find the average pressure
averagePressure = cal_average(pressure_prediction)
print("The average pressure is", averagePressure)

# scale pressure in range with the root key
num_notes = len(note_names)

pressureToNote = scale(averagePressure, pressureMin, pressureMax, 0, num_notes)

print("Atmospheric pressure to key note is : ", note_names[int(pressureToNote)])

The average pressure is 1018.3
Atmospheric pressure to key note is :  D


## `Map musical scales to temperature and humidity cases`

You can create some weather scenarios and match them with musical scales.

Its really up to you to deternine if a cold windy days sounds like a mixolydian scale.

The cases here are arbitrary, our purpose is to add some variation / randomization 

In [24]:
scaleToSet = iso.Scale.minorPenta

# find the average temperature and humidity 
averageTemp = cal_average(temp_prediction)
print("The average temperature is", averageTemp)
averageHum = cal_average(humidity_prediction)
print("The average humidity is", averageHum)


if averageTemp < 0 and averageHum < 30 :
  scaleToSet = iso.Scale.locrian
elif averageTemp > 10 and averageHum > 40 :
  scaleToSet = iso.Scale.maj7
elif averageTemp > 20 and averageHum > 60 :
  scaleToSet = iso.Scale.ritusen
elif averageTemp > 30 and averageHum > 30 :
  scaleToSet = iso.Scale.puremajor
elif averageTemp > 40 and averageHum > 60 :
  scaleToSet = iso.Scale.phrygian

print(scaleToSet)

The average temperature is 16.67525
The average humidity is 66.875
maj7 [0, 2, 4, 5, 7, 9, 10]


##`Finalize the root key and scale of the sequence`

In [25]:
key = iso.Key(int(pressureToNote), scaleToSet)

##`Match Wind speed the notes in the sequence`

Wind speed in measure in `m/s` 

`Beaufort Wind Scale`
- 0.5-1.5 m/s Smoke drifts with air, weather vanes inactive
- 3.5-5 m/s is a Gentle breeze
- 11-13.5 m/s Strong breeze
- 17-20 m/s Twigs broken off trees, walking against wind very difficult

In [26]:
minWindSpeed = 0 #0 --- Calm 	less than 1 mph (0 m/s)
maxWindSpeed = 12 #12 -- Hurricane 	over 73 mph
minNote= 0
maxNote = 8

sequenceNotes = []

# Scale to Sequence Durations
for speed in wind_speed_prediction:
  scaled_value = scale(speed,
                           int(minWindSpeed),
                           int(maxWindSpeed),
                           int(minNote),
                           int(maxNote))
  sequenceNotes.append(int(scaled_value))

# `Scale wind direction with Octaves`

In [27]:
minWindDir = 0
maxWindDir = 360
minOctave = 3
maxOctave = 8

octaves = []

# Scale to Sequence Durations
for direction in wind_degree_prediction:
  scaled_value = scale(direction,
                           minWindDir,
                           maxWindDir,
                           minOctave,
                           maxOctave)
  octaves.append(int(scaled_value))

# `Scale latitude and longitude to sequence amplitudes`

- latitude  ` -90 to 90`
- longtidude ` -180 to 180`

#`Create a timeline of 4 sequences`

In [81]:
import isobar as iso
from isobar.io import MidiFileOutputDevice
import sys
import random
import random

import logging
logging.basicConfig(level=logging.DEBUG, format="[%(asctime)s] %(message)s")

filename = "output.mid"

# MIDI Config
output = MidiFileOutputDevice(filename)
timeline = iso.Timeline(iso.MAX_CLOCK_RATE, output_device=output)
timeline.stop_when_done = True

####--- Process ---###


sequence = iso.PSequence(sequenceNotes, len(sequenceNotes))

#Returns a finite subsequence of an input pattern.
sequence = iso.PSubsequence(sequence, 2, 16)
sequence = iso.PPingPong(sequence,4)

sequenceShufle = iso.PShuffle(sequence)


timeline.schedule({
    "note": iso.PDegree(sequence, key),
    "octave": octaves[4],
    "gate": iso.PSequence([ 0.5, 1, 2, 1]),
    "amplitude": iso.PSequence([ 100, 80, 60, 40], 4),
    "duration": 1.0
})

timeline.schedule({
    "note": iso.PDegree(sequencePingPong, key),
    "octave": octaves[1],
    "gate": 1,
    "amplitude": iso.PSequence([ 80, 70, 60, 50], 4),
    "duration": 0.5
}, delay=0.5)


timeline.run()
output.write()

print("Wrote to output file: %s" % filename)
print(scaleToSet)

[2021-11-11 10:44:35,743] Timeline: Scheduled new track (total tracks: 1)
[2021-11-11 10:44:35,744] Timeline: Starting
[2021-11-11 10:44:35,747] --------------------------------------------------------------------------------
[2021-11-11 10:44:35,749] Tick (1 active tracks, 1 pending events)
[2021-11-11 10:44:35,757] Timeline: Scheduled new track (total tracks: 2)
[2021-11-11 10:44:35,763] --------------------------------------------------------------------------------
[2021-11-11 10:44:35,765] Tick (2 active tracks, 0 pending events)
[2021-11-11 10:44:35,776] Clock: Timer overflowed (late by 0.029s)
[2021-11-11 10:44:35,780] --------------------------------------------------------------------------------
[2021-11-11 10:44:35,781] Tick (2 active tracks, 0 pending events)
[2021-11-11 10:44:35,794] --------------------------------------------------------------------------------
[2021-11-11 10:44:35,795] Tick (2 active tracks, 0 pending events)
[2021-11-11 10:44:35,808] ------------------

Wrote to output file: output.mid
maj7 [0, 2, 4, 5, 7, 9, 10]


##`Get an instrument to play the result, then render to output`

In [82]:
#@title Default title text { vertical-output: true }
!apt install fluidsynth
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2

# render to wav
!fluidsynth -ni font.sf2 output.mid -F output.wav -r 44100

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fluidsynth is already the newest version (1.1.9-1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
FluidSynth version 1.1.9
Copyright (C) 2000-2018 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of E-mu Systems, Inc.

Rendering audio to file 'output.wav'..


##`Listen to it`

In [83]:
from IPython.display import Audio
Audio('output.wav')